In [105]:
import csv
import os
from functools import partial
import json

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import scipy as sp

from mpl_toolkits.mplot3d import Axes3D

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)
from gerrychain.metrics import efficiency_gap, mean_median, polsby_popper
from gerrychain.proposals import recom, flip
from gerrychain.updaters import cut_edges
from gerrychain.tree import recursive_tree_part

In [106]:
unique_label = "GEOID10"
pop_col = "TOTPOP"
district_col = "CD"

plot_path = "../DiffusionDistances/Data/PA_VTD.shp"
graph = Graph.from_json("../DiffusionDistances/Data/PA_VTD.json")

In [107]:
df = gpd.read_file(plot_path)

In [108]:
updaters = {
    "population": updaters.Tally("TOT_POP", alias="population"),
    "cut_edges": cut_edges, #"black_pop": updaters.Tally("BPOP", alias = "black_pop"),
    #"nh_white": updaters.Tally("NH_WHITE", alias = "nh_white"),
    #"nh_black": updaters.Tally("NH_BLACK", alias = "nh_black"),
    #"hisp": updaters.Tally("HISP", alias = "hisp"),
    #"vap": updaters.Tally("VAP", alias = "vap"),
    #"hvap": updaters.Tally("HVAP", alias = "hvap"),
    #"wvap": updaters.Tally("WVAP", alias = "wvap"),
    #"bvap":updaters.Tally("BVAP", alias = "bvap")
}

initial_partition = Partition(graph, "GOV", updaters)

In [110]:
num_dist = len(initial_partition)
ideal_population = sum(initial_partition["population"].values()) / len(
    initial_partition)
random_plan = recursive_tree_part(graph, range(num_dist), ideal_population, "TOT_POP", 0.01, 1)
base_partition = Partition(graph, random_plan, updaters)
random_plan_2 = recursive_tree_part(graph, range(num_dist), ideal_population, "TOT_POP", 0.01, 1)
new_partition = Partition(graph, random_plan_2, updaters)

In [111]:
# Given a base_partition with some labelling and a new_partition, we would like 
# the labelling of new_partition to be as close as possible to the base_partition.
# That is, we would like as many nodes as possible to remain in the same district.
# This function implements a greedy algorithm to do so (thus is not necessarily optimal).
# It outputs a dictionary that takes the labelling given by base_partition and 
# says which label in new_partition should be sent to that base_partition label.  
# It also outputs the number of displaced nodes
# (so the output is a tuple with first entry the dictionary, second entry number of
# displaced nodes)

def greedy_hamming(base_partition, new_partition):
    names = [j for j in base_partition.parts]
    new_names = {}
    for i in new_partition.parts:
        intersection_sizes = {}
        for name in names:
            intersection_sizes.update({len(set(base_partition.assignment.parts[name]).intersection(set(new_partition.assignment.parts[i]))): name})
        new_names.update({intersection_sizes[max(intersection_sizes.keys())]: i})
        names.remove(intersection_sizes[max(intersection_sizes.keys())])
    tot_nodes = len(new_partition.assignment)
    final_int_sizes = []
    for i in base_partition.parts:
        x = len(set(new_partition.assignment.parts[new_names[i]]).intersection(set(base_partition.assignment.parts[i])))
        final_int_sizes.append(x)
    ham_dist = tot_nodes - sum(final_int_sizes)
    return new_names, ham_dist;

In [112]:
# Given a base_partition with some labelling and a new_partition, we would like 
# the labelling of new_partition to be as close as possible to the base_partition.
# That is, we would like as many people as possible to remain in the same district.
# This function implements a greedy algorithm to do so (thus is not necessarily optimal).
# It outputs a dictionary that takes the labelling given by base_partition and 
# says which label in new_partition should be sent to that base_partition label.  It also outputs the number of displaced people
# (so the output is a tuple with first entry the dictionary, second entry number of
# displaced people)


def greedy_hamming_pop(base_partition, new_partition):
    names = [j for j in base_partition.parts]
    new_names = {}
    for i in new_partition.parts:
        intersections = {}
        intersection_pops = {}
        for name in names:
            intersections.update({name: set(base_partition.assignment.parts[name]).intersection(set(new_partition.assignment.parts[i]))})
            intersection_pops.update({sum([new_partition.graph.nodes[node]["TOTPOP"] for node in intersections[name]]): name})
        new_names.update({intersection_pops[max(intersection_pops.keys())]: i})
        names.remove(intersection_pops[max(intersection_pops.keys())])
    tot_pop = sum(base_partition["population"].values())
    final_int_pops = []
    for i in base_partition.parts:
        intersection_set = set(new_partition.assignment.parts[new_names[i]]).intersection(set(base_partition.assignment.parts[i]))      
        intersection_list = list(intersection_set)
        intersection_pops = sum([new_partition.graph.nodes[node]["TOTPOP"] for node in intersection_list])
        final_int_pops.append(intersection_pops)
    ham_dist_pop = tot_pop - sum(final_int_pops)    
    return new_names, ham_dist_pop;

In [115]:
def diffusion_distance_pre_sym(base_partition, new_partition):
    #build a graph and get dictionary from greedy_hamming algorithm of which districts correspond
    graph = base_partition.graph
    dist_list = list(base_partition.parts)
    dist_dict = greedy_hamming(base_partition,new_partition)[0]
    #This list holds the expected diffusion time for each district, averaged by
    # the number of nodes in the district
    diffusion_list = []
    #iterating over all districts
    for dist in dist_list:
        #grabbing the corresponding district in the new_partition
        dist2 = dist_dict[dist]
        #build the random walk matrix for the graph
        RW = (nx.adjacency_matrix(graph, weight = None)).astype(float)
        for i in range(len(graph.nodes)):
            RW[i,:] = RW[i,:]/len(list(graph.neighbors(i)))
        #We delete entries from RW; this helps track indices
        ref_list = list(graph.nodes)
        ref_list = np.delete(ref_list,np.array(list(new_partition.parts[dist2])),0)
        # Deleting these same entries from RW (by keeping the complement)
        to_keep = sorted(list(set([i for i in range(len(graph.nodes))]).difference(set(new_partition.parts[dist2]))))
        P = RW[to_keep,:][:,to_keep]
        M = sp.sparse.identity(len(ref_list))
        M = sp.sparse.csr_matrix(M)
        M = M - P
        uno = np.ones((len(ref_list),1))
        rowsum = sp.sparse.linalg.spsolve(M, uno)
        diff_nodes = list(set(base_partition.parts[dist]).difference(set(new_partition.parts[dist2])))
        k = 0
        for i in range(len(ref_list)):
            if ref_list[i] in set(diff_nodes):
                k += rowsum[i]
        # Dividing that sum by the number of nodes in the sum
        k = float(k)/len(diff_nodes)
        diffusion_list.append(k)
   
    return diffusion_list

# Function that measures the expected time it takes for random walks
# between hamming paired districts to terminate.
# Outputs a tuple.  First entry is a list that is the time (averaged over 
# number of nodes) in each district, second entry is the sum of those times.

def diffusion_distance(base_partition,new_partition):
    first_half = np.array(diffusion_distance_pre_sym(base_partition,new_partition))
    second_half = np.array(diffusion_distance_pre_sym(new_partition,base_partition))
    diffusion_list = first_half+second_half
    # Final total diffusion time
    total_diff_time = sum(diffusion_list)
    return diffusion_list,total_diff_time;